# Distributed training with Vertex Reduction server

In [1]:
import os
import pprint
import sys
import time

from google.cloud import aiplatform

## Set environment constants

In [2]:
PROJECT_ID = 'jk-mlops-dev'
REGION = 'us-central1'
STAGING_BUCKET = 'gs://jk-vertex-staging'

## Prepare and test a training container

### Create a Dockerfile

In [22]:
#BASE_IMAGE = 'tensorflow/tensorflow:2.5.0-gpu'
BASE_IMAGE = 'gcr.io/deeplearning-platform-release/tf2-gpu.2-5'
CUDA_VERSION='cuda-11-2'
#BASE_IMAGE = 'gcr.io/deeplearning-platform-release/base-cu110'
MODEL_GARDEN_VERSION = '2.5.0'
TRAIN_IMAGE = f'gcr.io/{PROJECT_ID}/model_garden'
TF_TEXT='2.5.0'


dockerfile = f'''
FROM {BASE_IMAGE}

RUN apt remove -y google-fast-socket && \
    echo "deb https://packages.cloud.google.com/apt google-fast-socket main" | tee /etc/apt/sources.list.d/google-fast-socket.list && \
    curl -s -L https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add - && \
    apt update && apt install -y google-reduction-server

RUN pip install tf-models-official=={MODEL_GARDEN_VERSION} tensorflow-text=={TF_TEXT}

WORKDIR /

# Copies the trainer code to the docker image.
COPY trainer /trainer

ENTRYPOINT ["python"]
CMD ["-c", "print('Hello')"]
'''

with open('Dockerfile', 'w') as f:
    f.write(dockerfile)

### Build a container image

In [23]:
! docker build -t {TRAIN_IMAGE} .

Sending build context to Docker daemon  477.2kB
Step 1/7 : FROM gcr.io/deeplearning-platform-release/tf2-gpu.2-5
 ---> 950969e5619c
Step 2/7 : RUN apt remove -y google-fast-socket &&     echo "deb https://packages.cloud.google.com/apt google-fast-socket main" | tee /etc/apt/sources.list.d/google-fast-socket.list &&     curl -s -L https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add - &&     apt update && apt install -y google-reduction-server
 ---> Running in 51d366690f7b


Reading package lists...
Building dependency tree...
Reading state information...
The following packages will be REMOVED:
  google-fast-socket
0 upgraded, 0 newly installed, 1 to remove and 0 not upgraded.
After this operation, 0 B of additional disk space will be used.
(Reading database ... 80237 files and directories currently installed.)
Removing google-fast-socket (0.0.3) ...
Processing triggers for libc-bin (2.27-3ubuntu1.4) ...
deb https://packages.cloud.google.com/apt google-fast-socket main
OK

### Push the container to Container Registry

In [24]:
! docker push {TRAIN_IMAGE}

Using default tag: latest
The push refers to repository [gcr.io/jk-mlops-dev/model_garden]

64075143: Preparing 
0a3b7f7b: Preparing 
b42ed173: Preparing 
464d3f17: Preparing 
daea14d2: Preparing 
b28de254: Preparing 
52e30556: Preparing 
fc085027: Preparing 
7d90a58d: Preparing 
285b3362: Preparing 
0730cb59: Preparing 
18de1f93: Preparing 
d1dfb5d0: Preparing 
686f5924: Preparing 
5de2196f: Preparing 
383a0e80: Preparing 
eaf882b2: Preparing 
2519572d: Preparing 
28de254: Waiting g 
a8806df6: Preparing 
2e30556: Waiting g 
d90a58d: Waiting g 
85b3362: Waiting g 
0a9a6a11: Preparing 
de2196f: Waiting g 
86f5924: Waiting g 
83a0e80: Waiting g 
31d2d72b: Preparing 
519572d: Waiting g 
bfba824: Waiting g 
49f5bf51: Preparing 
49af22b: Waiting g 
363f69f: Waiting g 
a9a6a11: Waiting g 
a3b7f7b: Pushed   210.9MB/208.2MBlatest: digest: sha256:c66574cfeb57e288642a55ffdcb77b86987755b6e22482f550d0560b0b14a532 size: 7673


## Submit Vertext Training jobs

### Define helper functions

In [12]:
def prepare_worker_pool_specs(
    image_uri,
    args,
    cmd, 
    replica_count=1,
    machine_type='n1-standard-4',
    accelerator_count=0,
    accelerator_type='ACCELERATOR_TYPE_UNSPECIFIED',
    reduction_server_count=0,
    reduction_server_machine_type='n1-standard-4',
    reduction_server_image_uri='gcr.io/cloud-aiplatform-restricted/reductionserver'
):

    if accelerator_count > 0:
        machine_spec = {
            'machine_type': machine_type,
            'accelerator_type': accelerator_type,
            'accelerator_count': accelerator_count,
        }
    else:
        machine_spec = {
            'machine_type': machine_type
        }
    
    container_spec = {
        'image_uri': image_uri,
        'args': args,
        'command': cmd,
    }
    
    chief_spec = {
        'replica_count': 1,
        'machine_spec': machine_spec,
        'container_spec': container_spec
    }

    worker_pool_specs = [chief_spec]
    if replica_count > 1:
        workers_spec = {
            'replica_count': replica_count - 1,
            'machine_spec': machine_spec,
            'container_spec': container_spec
        }
        worker_pool_specs.append(workers_spec)
        
    if reduction_server_count > 1:
        workers_spec = {
            'replica_count': reduction_server_count,
            'machine_spec': {
                'machine_type': reduction_server_machine_type,
            },
            'container_spec': {
                image_uri: reduction_server_image_uri
            }
        }
    
    return worker_pool_specs

### Prepare worker pool specification

In [25]:
MNLI_TRAIN_SPLIT = 'gs://jk-vertex-demos/datasets/MNLI/mnli_train.tf_record'
MNLI_VALID_SPLIT = 'gs://jk-vertex-demos/datasets/MNLI/mnli_valid.tf_record'
BERT_HUB_URL = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/4'

job_name = "JOB_{}".format(time.strftime("%Y%m%d_%H%M%S"))
output_dir = f'gs://jk-vertex-demos/jobs'
model_dir = f'{output_dir}/{job_name}/model'
tfhub_cache_dir = f'{output_dir}/tfhub-cache'
config_file = 'trainer/glue_mnli_matched.yaml'
mode = 'train_and_eval'
experiment = 'bert/sentence_prediction'

machine_type = 'n1-standard-8'
accelerator_count = 1
accelerator_type = 'NVIDIA_TESLA_T4'

train_steps = 400
steps_per_loop = 100
summary_interval = 100
validation_interval = 200
checkpoint_interval = 200

replica_count = 2
global_batch_size = 32
all_reduce_alg = 'nccl'
#strategy = 'mirrored'
strategy = 'multi_worker_mirrored'

reduction_server_count=0



params_override = [
    'task.train_data.input_path=' + MNLI_TRAIN_SPLIT,
    'task.validation_data.input_path=' + MNLI_VALID_SPLIT,
    'task.train_data.global_batch_size=' + str(global_batch_size),
    'task.validation_data.global_batch_size=' + str(global_batch_size),
    'task.hub_module_url=' + BERT_HUB_URL,
    'runtime.num_gpus=' + str(accelerator_count),
    'runtime.distribution_strategy=' + strategy,
    'runtime.all_reduce_alg=' + all_reduce_alg,
    'trainer.train_steps=' + str(train_steps),
    'trainer.steps_per_loop=' + str(steps_per_loop),
    'trainer.summary_interval=' + str(summary_interval),
    'trainer.validation_interval=' + str(validation_interval),
    'trainer.checkpoint_interval=' + str(checkpoint_interval),
]


cmd = [
    "python", "trainer/train.py"
]
args = [
    '--experiment=' + experiment,
    '--mode=' + mode,
    '--model_dir=' + model_dir,
    '--config_file=' + config_file,
    '--tfhub_cache_dir=' + tfhub_cache_dir,
    '--params_override=' + ','.join(params_override),
]

worker_pool_specs = prepare_worker_pool_specs(
    image_uri=TRAIN_IMAGE,
    args=args,
    cmd=cmd,
    replica_count=replica_count,
    machine_type=machine_type,
    accelerator_count=accelerator_count,
    accelerator_type=accelerator_type
)

pp = pprint.PrettyPrinter()
print(pp.pformat(worker_pool_specs))

[{'container_spec': {'args': ['--experiment=bert/sentence_prediction',
                              '--mode=train_and_eval',
                              '--model_dir=gs://jk-vertex-demos/jobs/JOB_20210610_153821/model',
                              '--config_file=trainer/glue_mnli_matched.yaml',
                              '--tfhub_cache_dir=gs://jk-vertex-demos/jobs/tfhub-cache',
                              '--params_override=task.train_data.input_path=gs://jk-vertex-demos/datasets/MNLI/mnli_train.tf_record,task.validation_data.input_path=gs://jk-vertex-demos/datasets/MNLI/mnli_valid.tf_record,task.train_data.global_batch_size=32,task.validation_data.global_batch_size=32,task.hub_module_url=https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/4,runtime.num_gpus=1,runtime.distribution_strategy=multi_worker_mirrored,runtime.all_reduce_alg=nccl,trainer.train_steps=400,trainer.steps_per_loop=100,trainer.summary_interval=100,trainer.validation_interval=200,trainer.checkpoi

### Submit and monitor the job

In [26]:
PROJECT_ID = 'jk-mlops-dev'
REGION = 'us-west1'
STAGING_BUCKET = 'gs://jk-vertex-staging'

aiplatform.init(
    project=PROJECT_ID,
    location=REGION,
    staging_bucket=STAGING_BUCKET
)

In [27]:
display_name = job_name

job = aiplatform.CustomJob(
    display_name=display_name,
    worker_pool_specs=worker_pool_specs,
)

job.run(sync=False)

INFO:google.cloud.aiplatform.jobs:Creating CustomJob
INFO:google.cloud.aiplatform.jobs:CustomJob created. Resource name: projects/895222332033/locations/us-west1/customJobs/3423333851127087104
INFO:google.cloud.aiplatform.jobs:To use this CustomJob in another session:
INFO:google.cloud.aiplatform.jobs:custom_job = aiplatform.CustomJob.get('projects/895222332033/locations/us-west1/customJobs/3423333851127087104')
INFO:google.cloud.aiplatform.jobs:View Custom Job:
https://console.cloud.google.com/ai/platform/locations/us-west1/training/3423333851127087104?project=895222332033
INFO:google.cloud.aiplatform.jobs:CustomJob projects/895222332033/locations/us-west1/customJobs/3423333851127087104 current state:
JobState.JOB_STATE_PENDING
INFO:google.cloud.aiplatform.jobs:CustomJob projects/895222332033/locations/us-west1/customJobs/3423333851127087104 current state:
JobState.JOB_STATE_PENDING
INFO:google.cloud.aiplatform.jobs:CustomJob projects/895222332033/locations/us-west1/customJobs/3423333

In [10]:
job.resource_name

'projects/895222332033/locations/us-west1/customJobs/7943259027146801152'

INFO:google.cloud.aiplatform.jobs:CustomJob projects/895222332033/locations/us-west1/customJobs/7943259027146801152 current state:
JobState.JOB_STATE_PENDING
INFO:google.cloud.aiplatform.jobs:CustomJob projects/895222332033/locations/us-west1/customJobs/7943259027146801152 current state:
JobState.JOB_STATE_PENDING
INFO:google.cloud.aiplatform.jobs:CustomJob projects/895222332033/locations/us-west1/customJobs/7943259027146801152 current state:
JobState.JOB_STATE_PENDING
INFO:google.cloud.aiplatform.jobs:CustomJob projects/895222332033/locations/us-west1/customJobs/7943259027146801152 current state:
JobState.JOB_STATE_PENDING
INFO:google.cloud.aiplatform.jobs:CustomJob projects/895222332033/locations/us-west1/customJobs/7943259027146801152 current state:
JobState.JOB_STATE_PENDING
INFO:google.cloud.aiplatform.jobs:CustomJob projects/895222332033/locations/us-west1/customJobs/7943259027146801152 current state:
JobState.JOB_STATE_PENDING


In [ ]:
job.wait()

### Test the container image locally

In [ ]:
MNLI_TRAIN_SPLIT = 'gs://jk-vertex-demos/datasets/MNLI/mnli_train.tf_record'
MNLI_VALID_SPLIT = 'gs://jk-vertex-demos/datasets/MNLI/mnli_valid.tf_record'
BERT_HUB_URL = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/4'

num_gpus = 2 
strategy = 'mirrored'
#strategy = 'multi_worker_mirrored'

params_override = [
    'task.train_data.input_path=' + MNLI_TRAIN_SPLIT,
    'task.validation_data.input_path=' + MNLI_VALID_SPLIT,
    'task.hub_module_url=' + BERT_HUB_URL,
    'runtime.num_gpus=' + str(num_gpus),
    'runtime.distribution_strategy=' + strategy,
]

params = ','.join(params_override)

In [ ]:
!docker run -it --rm --gpus all {TRAIN_IMAGE} trainer/train.py \
--experiment=bert/sentence_prediction \
--mode=train_and_eval \
--model_dir={STAGING_BUCKET}/test \
--config_file=trainer/glue_mnli_matched.yaml \
--params_override={params}  
